In [3]:
input_files = ['news_text_en_phrased.json']

import os
for root, dirs, files in os.walk('/home/qiuwenda/CP5/', topdown=False):
    for name in files:
        if name.endswith('news_text_en_phrased_append.json'):
            input_files.append(name)
print(input_files)


['news_text_en_phrased.json', 'news_text_en_phrased_append.json', 'week_14_news_text_en_phrased_append.json', 'week_15_news_text_en_phrased_append.json', 'week_16_news_text_en_phrased_append.json']


In [5]:

url2results = dict()
narratives = ['benefits/connections/afghanistan', 'benefits/covid', 'benefits/development/energy', 'benefits/development/maritime', 'benefits/development/roads', 'benefits/jobs', 'controversies/china/border', 'controversies/china/debt', 'controversies/china/exploitation', 'controversies/china/funding', 'controversies/china/naval', 'controversies/china/uighur', 'controversies/pakistan/army', 'controversies/pakistan/bajwa', 'controversies/pakistan/baloch', 'controversies/pakistan/students', 'leadership/bajwa', 'leadership/khan', 'leadership/sharif', 'opposition/kashmir', 'opposition/propaganda']
selected_nar = []
import json
from collections import defaultdict
import itertools
url2title = dict()

def open_all():
    global file_tmp
    file_tmp = [open(f'../{input_file}') for input_file in input_files]
    return itertools.chain.from_iterable(file_tmp)

def close_all():
    global file_tmp
    for i in file_tmp:
        i.close()

with open('WeSTClass11/news_manual/corpus.txt', 'w') as fout:
    for line in open_all():
        js = json.loads(line)
        text = ((js['phrased_title'] if js['phrased_title'] else ' ') + ' ' + (js['phrased_article'] if js['phrased_article'] else ' ')).replace('\r', ' ').replace('\n', ' ').strip()
        url2title[js['url']] = js['phrased_title']
        if len(text) > 10 and len(text.split(' ')) > 5:
            fout.write(' '.join(text.split(' ')[:480]) + '\n')
    close_all()

In [6]:
!rm -rf /home/qiuwenda/CP5/frame_classification/WeSTClass11/results/*
!rm -f /home/qiuwenda/CP5/frame_classification/WeSTClass11/news_manual/embedding
!rm -f /home/qiuwenda/CP5/frame_classification/WeSTClass11/news_manual/out.txt

In [7]:
!cd /home/qiuwenda/CP5/frame_classification/WeSTClass11/ && bash test.sh

2021-02-01 09:23:53.214826: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/qiuwenda/anaconda3/envs/west/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/qiuwenda/anaconda3/envs/west/lib/python3.8/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  wa

In [6]:
url2west = dict()
selected_nar = ['controversies/pakistan/students', 'leadership/sharif', 'leadership/bajwa', 'controversies/china/uighur', 'controversies/china/border', 'benefits/development/roads', 'controversies/pakistan/baloch', 'benefits/jobs', 'opposition/propaganda', 'benefits/development/energy', 'controversies/pakistan/bajwa']
A = ['bajwa', 'baloch', 'benefits', 'border', 'china', 'controversies', 'development', 'energy', 'jobs', 'leadership', 'opposition', 'pakistan', 'propaganda', 'roads', 'sharif', 'students', 'uighur', 'cpec']
B = [['gen_bajwa','qamar_javed_bajwa','gen_qamar_javed_bajwa'],['baloch','balochistan','balochi','baluch'],['win-win','benefit','benefits'],['siachen_glacier','karakoram_pass','boundary','borders','frontier','indo','along','lac','crossing','sikkim','line_of_actual_control'],['china','beijing','chinese','xi','taiwan','xi_jinping','chinas'],['backlash_against','controversy','rumors','rumours','speculation','backlash','bizarre_claim','conspiracy_theories','fake_news'],['construction','development','projects'],['dam','petroleum','gas','oil','natural_gas','power_generation','renewable_energy','clean_energy','crude_oil','fertilizer','mining'],['employment','jobs','pakistan'],['coalition','alliance','leader','factions','political','leaders','party','supporters'],['opposition_leader','opposition_parties'],['pakistan','pak','pm_imran','imran_khan','islamabad','pakistani'],['propaganda','disinformation','fake_news','misinformation','false_information'],['underpasses','underpass','mortal_way','expressway','bridge','bridges'],['shehbaz_sharif','shahbaz_sharif','nawaz_sharif'],['students','online_classes','classes','university','universities','school','schools','classmates'],['uighurs','uyghur','uyghurs','xinjiang','minority','racial','discrimination'],['cpec','china','pakistan','china-pakistan','economic_corridor','india']]
topic2keywords_manual = {i : j for i, j in zip(A, B)}
nar2key = {i: [j for j in i.split('/')] for i in narratives}
nar2keyweight = set()
import numpy as np
cnt = 0
valid_url = []
# with open(f'../{input_file}') as fin1:
fin1 = open_all()
for line in fin1:
    js = json.loads(line)
    text = ((js['phrased_title'] if js['phrased_title'] else ' ') + ' ' + (js['phrased_article'] if js['phrased_article'] else ' ')).replace('\r', ' ').replace('\n', ' ').strip()
    url2title[js['url']] = js['phrased_title']
    if len(text) > 10 and len(text.split(' ')) > 5:
        valid_url.append(js['url'])
close_all()

fin1 = open_all()
with open('WeSTClass11/news_manual/out.txt') as fin2:
    for u, line2 in zip(valid_url, fin2):
        classes = ['bajwa', 'baloch', 'benefits', 'border', 'china', 'controversies', 'development', 'energy', 'jobs', 'leadership', 'opposition', 'pakistan', 'propaganda', 'roads', 'sharif', 'students', 'uighur']
        raw = {i : j for i, j in zip(classes, list(map(float, line2.strip().split(','))))}
        west = dict()
        for nar in selected_nar:
            if nar in nar2keyweight:
                score = 1.0
                w = 0
                for key in nar2keyweight[nar]:
                    _s = raw[key]
                    score *= np.power(_s, nar2keyweight[nar][key])
                    w += nar2keyweight[nar][key]
                score = np.power(score, 1.0 / w)
            else:
                nk = topic2keywords_manual[nar] if nar in topic2keywords_manual else nar2key[nar]
                score = 1.0
                weight = [0.0 for i in range(len(nk))]
                weight[-1] += 0.8
                for i in range(len(nk) - 1):
                    weight[i] += 0.2 / (len(nk) - 1)
                for key, w in zip(nk, weight):
                    _s = raw[key]
                    score *= np.power(_s, w)
                score = np.power(score, 1.0 / sum(weight))
            west[nar] = score
        url2west[u] = west
        cnt += 1
    close_all()
print(cnt)

1909


In [7]:

chosen_url = set()
for nar in selected_nar:
    urls = list(url2west)
    urls = sorted(urls, key=lambda u:url2west[u][nar], reverse=True)
    for u in urls[:300]:
        chosen_url.add(u)

import json
keywords = json.load(open('keywords.txt'))
global_cnt = {"benefits/covid": 485344, "benefits/development/roads": 74575, "benefits/development/maritime": 44606, "benefits/development/energy": 55211, "leadership/bajwa": 18469, "opposition/propaganda": 179989, "opposition/kashmir": 484026, "benefits/jobs": 34899, "leadership/khan": 164996, "controversies/china/uighur": 49202, "controversies/china/border": 1280130, "controversies/china/debt": 55764, "controversies/china/exploitation": 118370, "controversies/pakistan/students": 10867, "controversies/pakistan/baloch": 172937, "controversies/pakistan/army": 82715, "benefits/connections/afghanistan": 5863, "leadership/sharif": 92467, "controversies/pakistan/bajwa": 15371, "controversies/china/funding": 1425, "controversies/china/naval": 256}

def string_match(text):
    nar_cnt = defaultdict(int)
    found = 0
    for nar in narratives:
        for key in keywords[nar]:
            if text.lower().find(key) != -1:
                nar_cnt[nar] += 1
                found += 1
    if found >= 1:
        return {i: nar_cnt[i] for i in narratives}
    else:
        return None
    
for u in url2west:
    string_match_result = string_match(url2title[u] if u in url2title else '')
    if u in chosen_url:
        for nar in narratives:
            if nar not in url2west[u]:
                url2west[u][nar] = 0.0
        if 'nar' in url2west[u]:
            url2west[u].pop('nar')
        if len(url2west[u]) != 21:
            print(len(url2west[u]), url2west[u])
            assert False
    else:
        url2west[u] = string_match_result
#         assert len(url2west[u]) == 21
        
with open('url2west_prob_append.json', 'w') as fout:
    for u in url2west:
        i = url2west[u]
        if i:
            fout.write(json.dumps({'url': u, 'prob': i}) + '\n')
        

In [8]:
keywords

{'benefits/connections/afghanistan': ['afghanistan',
  'afghan',
  'kabul',
  'taliban'],
 'benefits/covid': ['covid',
  'covid-19',
  'coronavirus',
  'covid_19',
  'pandemic',
  'covid19',
  'cases_rise',
  'lockdown'],
 'benefits/development/energy': ['renewable',
  'renewables',
  'energy_consumption',
  'renewable_energy',
  'power_generation',
  'coal_expansion',
  'energy_projects'],
 'benefits/development/maritime': ['maritime',
  'indian_ocean',
  'vessels',
  'coast',
  'maritime_security',
  'fleet',
  'naval',
  'indo_-_pacific',
  'coast_guard',
  'coastal',
  'maritime_domain',
  'south_china_sea',
  'arabian_sea'],
 'benefits/development/roads': ['mortal_way',
  'expressway',
  'highway',
  'underpass'],
 'benefits/jobs': ['employment',
  'jobs',
  'unemployment',
  'lose_jobs',
  'unemployed'],
 'controversies/china/border': ['india_china_border_conflict',
  'siachen_glacier',
  'karakoram_pass',
  'border',
  'casualties',
  'borders',
  'frontier',
  'lac',
  'crossin

In [57]:

# for nar in selected_nar:
#     urls = list(url2west)
#     urls = sorted(urls, key=lambda u:url2west[u][nar] if url2west[u] else 0.0, reverse=True)
#     for u in urls[:30]:
#         print(nar, url2title[u])


controversies/pakistan/students rock climbers rescale heights of satisfaction as tourists return
controversies/pakistan/students maya moore 's help to win jonathan Irons 's freedom shows women leading change
controversies/pakistan/students black wine professionals demand to be seen
controversies/pakistan/students ' where are you really from ? ' : growing up black in ireland
controversies/pakistan/students india accounts for nearly_one_third of the world 's missing women : united_nations
controversies/pakistan/students young hongkonger becomes first person charged under national_security_law       eng_news   news   20200703
controversies/pakistan/students this rocket scientist is tracing black ingenuity through barbecue
controversies/pakistan/students a hard conversation for the latino community
controversies/pakistan/students jk easy Copier 70 gsm A4 500 sheets Copier paper box ( 5 reams ) : amazon.in : office products
controversies/pakistan/students review : netflix 's baby - Sitters 